## Air-Quality-in-Durban-Part-1-EDA-with-MongoDB by Dumisani Maxwell Mukuchura

#### Dataset Source: *https://open.africa/dataset/sensorsafrica-airquality-archive-durban*

##### Contact: dumisanimukuchura@gmail.com | LinkedIn: https://www.linkedin.com/in/dumisani-maxwell-mukuchura-4859b7170/

##### This Project will explore the time series dataset for Air Quality and will dissect it with Exploratory Data Analysis 

Kindly note this was done from a Local machine on Windows with a locally hosted database using Mongo DB, Mongo DB Compass and MongoDB Shell i.e a NoSQL database.  

In [2]:
#Import Libraries

from pprint import PrettyPrinter  #makes documents easier to read
import pandas as pd 
from pymongo import MongoClient  #useful to get Data from Mongo DB   

## 1. Prepare Data

### 1.1. Connect

In [6]:
pp = PrettyPrinter(indent = 2)
client = MongoClient(host= "localhost", port = 27017)

### 1.2. Explore

In [9]:
# What Databases are available?
pp.pprint(list(client.list_databases()))

[ {'empty': False, 'name': 'admin', 'sizeOnDisk': 40960},
  {'empty': False, 'name': 'air_quality', 'sizeOnDisk': 49152},
  {'empty': False, 'name': 'config', 'sizeOnDisk': 110592},
  {'empty': False, 'name': 'local', 'sizeOnDisk': 40960}]


In [10]:
#Assign database to db variable
db = client["air_quality"]

In [11]:
# View available collections in the db

list(db.list_collections())[0]

{'name': 'durban',
 'type': 'collection',
 'options': {},
 'info': {'readOnly': False,
  'uuid': Binary(b'\xea\x8d\xa6{q\x94@\xed\xbc\xa5\xd8\x83\xdb\x00;\xcb', 4)},
 'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}

In [12]:
# View available collections in the db

list(db.list_collection_names())

['durban']

In [ ]:
# Assign the Durban collection to durban variable